# This program connects to Box.com, gets all of the folders and images and stores information into the database.

https://developer.box.com/en/guides/authentication/oauth2/with-sdk/

In [1]:
!pip install boxsdk
!pip install auth
!pip install redis
!pip install mysql.connector
!pip install requests

  Using cached boxsdk-2.7.1-py2.py3-none-any.whl (126 kB)
  Using cached requests_toolbelt-0.9.1-py2.py3-none-any.whl (54 kB)
  Using cached requests-2.22.0-py2.py3-none-any.whl (57 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
     |████████████████████████████████| 125 kB 3.4 MB/s eta 0:00:01
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached auth-0.5.3-py3-none-any.whl (14 kB)
     |████████████████████████████████| 4.6 MB 3.5 MB/s eta 0:00:01     |█████████████████▋              | 2.5 MB 3.5 MB/s eta 0:00:01
  Using cached eventlet-0.25.1-py2.py3-none-any.whl (222 kB)
  Using cached gunicorn-20.0.4-py2.py3-none-any.whl (77 kB)
  Using cached mongoengine-0.19.1.tar.gz (157 kB)
  Using cached blinker-1.4.tar.gz (111 kB)
     |████████████████████████████████| 41 kB 274 kB/s  eta 0:00:01
  Using cached dnspython-1.16.0-py2.py3-none-any.whl (188 kB)
  Using cached monotonic-1.5-py2.py3-none-any.whl (5.3 kB)
     |████████████████████████████████| 460 k

# Run the cell below to authenticate with Box.com so we can use the API

In [1]:
from boxsdk import OAuth2

import json
#Set the file we want to use for authenticating a Box app
#The json file stores the client_id and client_secret so we don't have it in the code.
# The json file looks like this:
#{
#"client_id":"___the_codes_for_client_id___",
#"client_secret":"___the_codes_for_client_secret___"
#}

oauth_settings_file = '/home/student/jeff_not_data_5000/programs/box_app_test.json'
with open(oauth_settings_file, "r") as read_file:
    oauth_data = json.load(read_file)
print(oauth_data["client_id"])
print(oauth_data["client_secret"])

oauth = OAuth2(
    client_id=oauth_data["client_id"],
    client_secret=oauth_data["client_secret"]
)

auth_url, csrf_token = oauth.get_authorization_url('https://jeffblackadar.ca')
print("Click on this:")
print(auth_url)
print(csrf_token)
print("Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.")


rvy5o5ic72c00ldi8vpcqmof0t7h6w85
VWJQ1LTracitLw5HfG8Q0TKplBogFEIT
Click on this:
https://account.box.com/api/oauth2/authorize?state=box_csrf_token_uMjGaEk44cVQaueo&response_type=code&client_id=rvy5o5ic72c00ldi8vpcqmof0t7h6w85&redirect_uri=https%3A%2F%2Fjeffblackadar.ca
box_csrf_token_uMjGaEk44cVQaueo
Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.


In [2]:
from boxsdk import Client

# Make sure that the csrf token you get from the `state` parameter
# in the final redirect URI is the same token you get from the
# get_authorization_url method to protect against CSRF vulnerabilities.
#assert 'THE_CSRF_TOKEN_YOU_GOT' == csrf_token
access_token, refresh_token = oauth.authenticate('b38rXomqkDGo52bIWyaATtepFkRccqA1')
client = Client(oauth)

## Test the connection to Box.com - list all folders and files using recursion

In [ ]:
def process_subfolder_test(client, folder_id, folder_name):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder_test(client, item.id,folder_name+"/"+item.name)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))            
    return

process_subfolder_test(client, '0',"")

## Start MySQL

sudo systemctl start mysql

## Get all folders and store all files into a database.

In [17]:
def process_subfolder(client, folder_id, folder_name,cursor_insert):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder(client, item.id,folder_name+"/"+item.name,cursor_insert)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))
            
            insert_image = ("INSERT INTO tbl_box_images (`id_box_file`,`id_box_folder`,`folder`,`file_name`,`img_src`,`img_url`) VALUES (%s,%s,%s,%s,%s,%s);")
            
            cursor_insert.execute(insert_image, (item.id, folder_id, folder_name, item.name, folder_name+"/"+item.name, ""))
            mariadb_connection.commit()
    return





In [ ]:
import requests
import shutil
from pathlib import Path
base_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\pompeii\\box'


import mysql.connector as mariadb

mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")

cursor_insert = mariadb_connection.cursor(buffered=True)

process_subfolder(client, '0',"", cursor_insert)
    
mariadb_connection.close()    


In [20]:
mariadb_connection.close()    

## Process images: Connect to Box.com, download image, get its hash value.

In [6]:
 !pip install imutils

  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25853 sha256=60a616b9e2cabad44117bf93922a209f14a7f5a84c2998c601ce231cfe3d5d4c
  Stored in directory: /home/student/.cache/pip/wheels/27/4d/3a/6dcdf7c3ebc87bf1ae013d96c9cf060ccfe334bb5ee769f377
Successfully built imutils


In [3]:
# Image hash functions
# import the necessary packages
# Credit Adrian Rosebrock https://www.pyimagesearch.com
from imutils import paths
import argparse
import time
import sys
import cv2
import os

# hashing with OpenCV and PythonPython
def dhash(image, hashSize=8):
    # resize the input image, adding a single column (width) so we
    # can compute the horizontal gradient
    resized = cv2.resize(image, (hashSize + 1, hashSize))

    # compute the (relative) horizontal gradient between adjacent
    # column pixels
    diff = resized[:, 1:] > resized[:, :-1]

    # convert the difference image to a hash
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])


In [6]:
download_process_box_thumbnail(client,3063, "580416299144", "/pinp5/Maps/Maps", "Plan Bay of Naples 1865 from Karl von Spruner’s 1865 rendering of Southern Italy and Sicily in antiquity Wikimedia.jpg", cursor_update)

NameError: name 'cursor_update' is not defined

In [4]:
import os
import shutil
from pathlib import Path
base_dir = '/home/student/jeff_not_data_5000/images/box/'

# using the thumbnail for image hashing does not work   
def download_process_box_full_id_box_file(client, id_box, id_box_file, folder, file_name, cursor_update):
    #Some file names can't be processed.  This uses the box file name instead.
    print("this file: "+id_box_file+" "+file_name)
    #using the the large 320 pixel thumbnail produces different results than the full image - it does not work
    #try:
    #file_info = client.file(id_box_file).get()
    file_name = id_box_file+".jpg"
    imagecontent = client.file(id_box_file).content()
    # load the image for the image hash later.

    img_local_path=base_dir+folder+"/"+file_name
    #print(img_local_path)
    img_file_name = file_name
    img_local_folder = base_dir+folder

    #see if the image is there already
    if not os.path.exists(img_local_path):
        img_file_name = file_name
        img_local_folder = base_dir+folder
        
        if not os.path.exists(img_local_folder):
            #os.mkdir(img_local_folder)
            path = Path(img_local_folder)
            path.mkdir(parents=True,exist_ok=True)
            #print(img_local_folder+'/'+ img_file_name)
            
            # Open the url image, set stream to True, this will return the stream content.
            # Open a local file with wb ( write binary ) permission.
            #local_file = open(img_local_folder+'/'+ img_file_name, 'wb')
        with open(img_local_folder+'/'+ img_file_name,'wb') as f:
                f.write(imagecontent)
            #print(thumbnail)
        del imagecontent
        f.close()
            #image hash
    imageHash = 0
    #print("image hash")
    if not os.path.exists(img_local_path):
        print("Error image not loaded for image hash   " + img_local_path)
    else:         
        # load the image from disk
        path = Path(img_local_folder)
        os.chdir(path)
        hash_image = cv2.imread(img_file_name)
 
        # if the image is None then we could not load it from disk (so skip it)
        if not hash_image is None:
            # convert the image to grayscale and compute the hash
            hash_image = cv2.cvtColor(hash_image, cv2.COLOR_BGR2GRAY)
            imageHash = dhash(hash_image)
            #print(imageHash)
        else:
            imageHash = 0
            #print("image hash - can't find image?")
        update_image = ("UPDATE tbl_box_images SET image_hash = %s WHERE id = %s;")
        print(id_box, imageHash)
        cursor_update.execute(update_image, (imageHash, id_box))
        mariadb_connection.commit()
        
        
        os.remove(img_local_path)
    return
    #except:
    #    print(sys.exc_info()[0])
    #    return
    

In [5]:
number_of_rows_to_run = 100000
#165806
start_id = 208587
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='/home/student/jeff_not_data_5000/programs/webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_update = mariadb_connection.cursor(buffered=True)
cursor.execute("SELECT id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src FROM webpage_images.tbl_box_images WHERE id>=%s AND id<%s", (start_id, end_id))
for id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src in cursor:
    #print(id, id_box_file, img_path_match, id_tbl_webpage_images, img_src)
    #don't include .xml, .htm* or .thmx
    #for row 158348
    if(img_path_match is None):
        img_path_match = ""
    if (img_src.lower().find(".xml")<0) and (img_src.lower().find(".htm")<0) and (img_src.lower().find(".thmx")<0) and (img_src.lower().find(".js")<0) and (img_src.lower().find(".pdf")<0) and (img_src.lower().find(".zip")<0) and (img_src.lower().find(".asp")<0) and (img_src.lower().find(".txt")<0) and (img_src.lower().find(".ico")<0):
        download_process_box_full_id_box_file(client, id, str(id_box_file), folder, file_name, cursor_update)
            
cursor.close()
cursor_update.close()
mariadb_connection.close()    


205436 17411737166915154629
this file: 411574478585 pompeiiinpictures 200903 Card 14 1008.jpg
205437 14561994968023020576
this file: 411574231549 pompeiiinpictures 200903 Card 14 1009.jpg
205438 6227638396658518425
this file: 411559060565 pompeiiinpictures 200903 Card 14 101.jpg
205439 2976231960766778781
this file: 411558726755 pompeiiinpictures 200903 Card 14 1010.jpg
205440 5399089610740116061
this file: 411564824680 pompeiiinpictures 200903 Card 14 1011.jpg
205441 12721094782915794233
this file: 411576675023 pompeiiinpictures 200903 Card 14 1012.jpg
205442 7697331649682383593
this file: 411566507343 pompeiiinpictures 200903 Card 14 1013.jpg
205443 6069068755155305350
this file: 411574572019 pompeiiinpictures 200903 Card 14 1014.jpg
205444 15373785959423291392
this file: 411558995038 pompeiiinpictures 200903 Card 14 1015.jpg


"GET https://api.box.com/2.0/files/411563027834/content" 401 0
{'Date': 'Tue, 18 Feb 2020 12:21:02 GMT', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000', 'WWW-Authenticate': 'Bearer realm="Service", error="invalid_token", error_description="The access token provided is invalid."', 'BOX-REQUEST-ID': '01a3a04e8f5d8703caf4e118632fb0bcc'}
b''



205445 12509074708026011062
this file: 411563027834 pompeiiinpictures 200903 Card 14 1016.jpg
205446 10380876787400586075
this file: 411560423831 pompeiiinpictures 200903 Card 14 1017.jpg
205447 632565343468912645
this file: 411578098080 pompeiiinpictures 200903 Card 14 1018.jpg
205448 11270506151938710027
this file: 411574262948 pompeiiinpictures 200903 Card 14 1019.jpg
205449 3341434868300803413
this file: 411558559758 pompeiiinpictures 200903 Card 14 102.jpg
205450 4135299762596236225
this file: 411562177829 pompeiiinpictures 200903 Card 14 1020.jpg
205451 2626242712518898636
this file: 411580107959 pompeiiinpictures 200903 Card 14 1021.jpg
205452 3515240810147359236
this file: 411573958041 pompeiiinpictures 200903 Card 14 1022.jpg
205453 3299571646228473358
this file: 411552019196 pompeiiinpictures 200903 Card 14 1023.jpg
205454 11829182868253393953
this file: 411561665657 pompeiiinpictures 200903 Card 14 1024.jpg
205455 13954977014859254544
this file: 411564665734 pompeiiinpicture

205533 1173407900080181482
this file: 411569715868 pompeiiinpictures 200903 Card 14 1096.jpg
205534 6000072948361890725
this file: 411564748745 pompeiiinpictures 200903 Card 14 1097.jpg
205535 9986367785239942566
this file: 411551985888 pompeiiinpictures 200903 Card 14 1098.jpg
205536 5735225866207386134
this file: 411550671520 pompeiiinpictures 200903 Card 14 1099.jpg
205537 5009636477771967854
this file: 411562526488 pompeiiinpictures 200903 Card 14 110.jpg
205538 1609272041913012644
this file: 411551623825 pompeiiinpictures 200903 Card 14 1100.jpg
205539 10629668260163697302
this file: 411574816319 pompeiiinpictures 200903 Card 14 1101.jpg
205540 6299810550827928939
this file: 411560787786 pompeiiinpictures 200903 Card 14 1102.jpg
205541 14238500953032723176
this file: 411552329391 pompeiiinpictures 200903 Card 14 1103.jpg
205542 11036117432343565684
this file: 411559011179 pompeiiinpictures 200903 Card 14 1104.jpg
205543 11831004681445595845
this file: 411573750203 pompeiiinpicture

205621 10766162005400640512
this file: 411560358390 pompeiiinpictures 200903 Card 14 1174.jpg
205622 8370126955293263761
this file: 411577148450 pompeiiinpictures 200903 Card 14 1175.jpg
205623 5383655837573611520
this file: 411575486706 pompeiiinpictures 200903 Card 14 1176.jpg
205624 12513652977972163858
this file: 411561439872 pompeiiinpictures 200903 Card 14 1177.jpg
205625 17128427144622217372
this file: 411557966676 pompeiiinpictures 200903 Card 14 1178.jpg
205626 12507776255475152140
this file: 411558926511 pompeiiinpictures 200903 Card 14 1179.jpg
205627 6445050451941012139
this file: 411579150326 pompeiiinpictures 200903 Card 14 118.jpg
205628 13417586089075554986
this file: 411577201506 pompeiiinpictures 200903 Card 14 1180.jpg
205629 6212483776733713848
this file: 411575586902 pompeiiinpictures 200903 Card 14 1181.jpg
205630 8517817801152006604
this file: 411550020167 pompeiiinpictures 200903 Card 14 1182.jpg
205631 18080464750351312954
this file: 411563960737 pompeiiinpictu

...

208580 3372719458554916358
this file: 411568788099 pompeiiinpictures 200903 Card 16 234.jpg
208581 6274525115600980842
this file: 411605078815 pompeiiinpictures 200903 Card 16 235.jpg
208582 14276779586508630701
this file: 411577452623 pompeiiinpictures 200903 Card 16 236.jpg
208583 13320607926744025965
this file: 411578408102 pompeiiinpictures 200903 Card 16 237.jpg
208584 7231172352915294281
this file: 411571689883 pompeiiinpictures 200903 Card 16 238.jpg
208585 12625606995816684939
this file: 411594233008 pompeiiinpictures 200903 Card 16 239.jpg
208586 10889401557028677130
this file: 411580482743 pompeiiinpictures 200903 Card 16 240.jpg
208587 8525959943525077816
this file: 411567037480 pompeiiinpictures 200903 Card 16 241.jpg


"GET https://api.box.com/2.0/files/411562710076/content" 401 0
{'Date': 'Wed, 19 Feb 2020 00:21:59 GMT', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000', 'WWW-Authenticate': 'Bearer realm="Service", error="invalid_token", error_description="The access token provided is invalid."', 'BOX-REQUEST-ID': '17a89694d65052c54820d43a04e4d0724'}
b''



208588 6009261689593346424
this file: 411562710076 pompeiiinpictures 200903 Card 16 242.jpg
208589 13138872827565251558
this file: 411561425950 pompeiiinpictures 200903 Card 16 243.jpg
208590 2136783428586973542
this file: 411553024789 pompeiiinpictures 200903 Card 16 244.jpg


Request "GET https://api.box.com/2.0/files/411553024789/content" failed with ConnectionError exception: ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',)),)


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [ ]:
mariadb_connection.close()    